In [1]:
from transformers import set_seed, BertTokenizer, TFAutoModelForMaskedLM, AutoTokenizer, get_scheduler, BertForPreTraining, BertForMaskedLM, BertConfig
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from torch.optim import AdamW
import argparse, torch, datasets, ast, operator, gc, os
import torch.nn as nn

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description='search for best template according to dev set')
parser.add_argument('--num_example', default=16, type=int, help="subsample num examples per class")
parser.add_argument('--batch_size', default=2, type=int, help="batch size")
parser.add_argument('--task', default='douban', type=str, help="task name")
parser.add_argument('--datasets', default='./datasets/', type=str, help="dataset dir")
args = parser.parse_args(args=[])

print(args)

Namespace(batch_size=2, datasets='./datasets/', num_example=16, task='douban')


In [3]:
pd_all = pd.read_csv(f'{args.datasets}{args.task}_output.csv',names=['labels','text'],header=0)

In [4]:
# create fewshot training set
for seed in range(5):
    set_seed(seed)
    pd_pos = pd_all[pd_all.labels==1].sample(16)
    pd_neg = pd_all[pd_all.labels==0].sample(16)
    pd_train = pd.concat([pd_pos,pd_neg])
    pd_train = pd_train.sample(frac=1)
    pd_train.to_csv(f'{args.datasets}{args.task}_output_{seed}.csv')

In [5]:
fewshot_train_datasets = []
for seed in range(5): 
    pd_train = pd.read_csv(f'{args.datasets}{args.task}_output_{seed}.csv',names=['labels','text'],header=0)
    fewshot_train_datasets.append(pd_train)

fewshot_train_pd = pd.concat(fewshot_train_datasets)
fewshot_train_pd = fewshot_train_pd.drop_duplicates()


In [6]:
len(fewshot_train_pd),len(pd_all)

(150, 1094)

In [7]:
# create fewshot dev set
pd_all = pd_all.drop(fewshot_train_pd.index)
pd_pos = pd_all[pd_all.labels==1].sample(16)
pd_neg = pd_all[pd_all.labels==0].sample(16)
pd_dev = pd.concat([pd_pos,pd_neg])
pd_dev = pd_dev.sample(frac=1)
pd_dev.to_csv(f'{args.datasets}{args.task}_output_dev.csv')

In [8]:
len(pd_dev),len(pd_all)

(32, 944)

In [9]:
# create fewshot test set
pd_all = pd_all.drop(pd_dev.index)
pd_all.to_csv(f'{args.datasets}{args.task}_output_test.csv')
len(pd_all)

912